Make that we are running the correct version of TensorFlow first

In [1]:
import tensorflow as tf
tf.__version__

/home/jupyter/.local/share/virtualenvs/line-reader-3vKrZmrE/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/share/virtualenvs/line-reader-3vKrZmrE/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/share/virtualenvs/line-reader-3vKrZmrE/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtyp

'2.0.0-beta1'

In [2]:
import sys

assert sys.version_info >= (3, 6) # Python ≥3.6 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

# Build a lines segmentation model based on unet architecture 

Here is [the paper](https://arxiv.org/pdf/1505.04597.pdf)

In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')

from pathlib import Path
Path.ls = lambda x: list(x.iterdir())  # Source: https://github.com/fastai/fastai/blob/master/fastai/core.py#L236

from tqdm import tqdm

from PIL import Image

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical

In [4]:
from recognizer.datasets import Dataset
from recognizer.networks import unet
from recognizer.networks import unet_mini
from recognizer.networks import fcn
from recognizer.networks import NetworkInput

In [5]:
lines_detector_path = Dataset.processed_data_path()/"lines_detector"

lines_detector_train_images_path = lines_detector_path/"train"/"images"
lines_detector_train_masks_path = lines_detector_path/"train"/"masks"

lines_detector_valid_images_path = lines_detector_path/"valid"/"images"
lines_detector_valid_masks_path = lines_detector_path/"valid"/"masks"

lines_detector_test_images_path = lines_detector_path/"test"/"images"
lines_detector_test_masks_path = lines_detector_path/"test"/"masks"

Hyperparameters

In [6]:
batch_size = 8
epochs = 16

number_of_classes = 3

In [7]:
model_save_path = Path("../recognizer/weights/lines_detector.h5")

## Load and augment the dataset

In [8]:
data_gen_args = dict(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_image_datagen = ImageDataGenerator(
    **data_gen_args,
    rescale=1/255
)
test_image_datagen = ImageDataGenerator(rescale=1/255)

mask_datagen = ImageDataGenerator(
    **data_gen_args,
    rescale=1/127
)

Create the generators

In [9]:
def create_generator_from_directory(image_datagen: ImageDataGenerator, mask_datagen: ImageDataGenerator, images_path: Path, masks_path: Path, seed: int = 1):
    print(len(images_path.ls()))
    image_generator = image_datagen.flow_from_directory(
        str(images_path),
        color_mode="grayscale",
        class_mode=None,
        target_size=(256, 256),
        batch_size=batch_size,
        seed=seed)
    mask_generator = mask_datagen.flow_from_directory(
        str(masks_path),
        color_mode="grayscale",
        class_mode="categorical",
        target_size=(256, 256),
        batch_size=batch_size,
        seed=seed)

    return zip(image_generator, mask_generator)

In [11]:
train_generator = create_generator_from_directory(train_image_datagen, mask_datagen, lines_detector_train_images_path, lines_detector_train_masks_path)
valid_generator = create_generator_from_directory(test_image_datagen, mask_datagen, lines_detector_valid_images_path, lines_detector_valid_masks_path)
test_generator = create_generator_from_directory(test_image_datagen, mask_datagen, lines_detector_test_images_path, lines_detector_test_masks_path)

986
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
246
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
307
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [ ]:
train_length = len(lines_detector_train_images_path.ls())
valid_length = len(lines_detector_valid_images_path.ls())

train_length, valid_length

In [ ]:
steps_per_epoch = train_length // batch_size
validation_steps = valid_length // batch_size

steps_per_epoch, validation_steps

## Build the model

In [ ]:
input_shape = (256, 256, 1)
# model = unet_mini(
# model = unet(
model = fcn(
    NetworkInput(input_shape=input_shape, number_of_classes=number_of_classes)
)

model.summary()

## Train the model

Check progress in TensorBoard by running tensorboard --logdir=logs/lines_detector_model in terminal

In [ ]:
# Try removing the old logs first
!rm -r ../logs/lines_detector_model

callbacks = [
    keras.callbacks.TensorBoard(log_dir='../logs/lines_detector_model')
]

In [ ]:
model.compile(optimizer=Adam(lr=1e-2),  # RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
for index, (images, masks) in enumerate(train_generator):
    if index >= steps_per_epoch:
        
        break
    pass

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
#     callbacks=callbacks,
#     validation_data=valid_generator,
#     validation_steps=validation_steps
)
history.history.keys()

In [ ]:
fig = plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.show()

# Save the model

In [ ]:
model.save(model_save_path)

## Restore the model

In [ ]:
model = keras.models.load_model(model_save_path)

model.trainable = False
model.compile(optimizer=Adam(lr=1e-2),  # RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.trainable_variables

## Predict on one image

In [ ]:
image_array = x[0]
test_batch = image_array.reshape((1,) + image_array.shape)

preds = model.predict(test_batch)
preds.shape

In [ ]:
# Merge the 3 channel classes into one for color
pred_image_array = np.expand_dims(np.argmax(preds[0], axis=2), axis=-1)
array_to_img(pred_image_array)

In [ ]:
np.min(preds[0]), np.max(preds[0])

In [ ]:
np.min(pred_image_array), np.max(pred_image_array)

In [ ]:
np.argmax(preds[0], axis=1).shape

In [ ]:
np.argmax(preds[0], axis=2).shape

In [ ]:
np.expand_dims(np.argmax(preds[0], axis=2), axis=-1).shape

In [ ]:
pred_image_array.shape

Check predication array

In [ ]:
for row in preds[0]:
    if np.max(row) > 0:
        print(row)

In [ ]:
np.argmax(preds[0], axis=2).shape

In [ ]:
for row in np.argmax(preds[0], axis=2):
    if np.max(row) > 0:
        print(row)
    else:
        print(">> EMPTY ROW <<")

Check target array

In [ ]:
for row in y[0]:
    if np.max(row) > 0:
        print(row)

In [ ]:
np.argmax(y[0], axis=2).shape

In [ ]:
for row in np.argmax(y[0], axis=2):
    if np.max(row) > 0:
        print(row)
    else:
        print(">> EMPTY ROW <<")